In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('C:/Users/HP/Desktop/data science files/gas_turbines.csv')

In [3]:
data.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


In [4]:
data.isna().sum()

AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TIT     0
TAT     0
TEY     0
CDP     0
CO      0
NOX     0
dtype: int64

In [5]:
from sklearn import preprocessing

In [8]:
scaler=preprocessing.StandardScaler()
data_sca=scaler.fit_transform(data)

In [9]:
data_scaler=pd.DataFrame(data_sca,columns=data.columns)

In [10]:
data_scaler

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,-1.439778,-0.826644,1.281436,-0.921232,-1.379101,-1.488376,0.585240,-1.231172,-1.357331,0.532012,1.387845
1,-1.449601,-0.748647,1.304564,-0.921495,-1.363528,-1.482325,0.585240,-1.229909,-1.363676,0.568733,1.393002
2,-1.434721,-0.686250,1.219086,-0.944385,-1.351309,-1.476275,0.568715,-1.230541,-1.360957,0.552938,1.363586
3,-1.413702,-0.623853,1.169060,-0.946884,-1.348194,-1.464173,0.583969,-1.229909,-1.356424,0.548933,1.382878
4,-1.368693,-0.545857,1.161883,-0.924389,-1.354663,-1.458123,0.582698,-1.229909,-1.350985,0.574179,1.348591
...,...,...,...,...,...,...,...,...,...,...,...
15034,-1.153182,-1.185428,1.401860,-0.865850,-1.498657,-2.063184,0.103453,-1.426381,-1.543161,1.145792,1.085751
15035,-1.303986,-1.138630,1.447753,-0.913470,-1.438759,-2.268905,-0.276638,-1.415642,-1.513247,1.293578,1.119943
15036,-1.386267,-1.076233,1.476971,-0.951488,-1.410967,-2.789257,-1.026650,-1.516089,-1.467922,2.695925,2.170062
15037,-1.420423,-0.998236,1.441590,-0.988848,-1.447624,-2.456474,-0.528337,-1.481343,-1.422598,1.924683,2.391165


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x=data_scaler.drop('TEY',axis=1)
y=data_scaler['TEY']

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=23)

In [14]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((12031, 10), (3008, 10), (12031,), (3008,))

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score,KFold
from keras.layers import Dropout

In [40]:

def create_model(learning_rate,dropout_rate,activation_function,init):
    model = Sequential()
    model.add(Dense(11,input_dim = 11,kernel_initializer = init,input_shape=[10],activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(6,kernel_initializer = init,input_shape=[10],activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = activation_function))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'mse',optimizer = adam,metrics = ['accuracy'])
    return model


model = KerasRegressor(build_fn = create_model,verbose = 0)



batch_size = [10,20,40]
epochs = [10,50,100]
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']
p_dstr = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init)



grid = RandomizedSearchCV(estimator = model,param_distributions = p_dstr,cv = KFold(),verbose = 10)
ran_result = grid.fit(x_train,y_train)


C:\Users\HP\AppData\Local\Temp\ipykernel_8944\457156793.py:15: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn = create_model,verbose = 0)
C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START activation_function=softmax, batch_size=10, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01
[CV 1/5; 1/10] END activation_function=softmax, batch_size=10, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01;, score=-1.979 total time= 2.3min
[CV 2/5; 1/10] START activation_function=softmax, batch_size=10, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01


C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


[CV 2/5; 1/10] END activation_function=softmax, batch_size=10, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01;, score=-1.933 total time= 2.1min
[CV 3/5; 1/10] START activation_function=softmax, batch_size=10, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01


C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


[CV 4/5; 10/10] END activation_function=linear, batch_size=10, dropout_rate=0.0, epochs=10, init=zero, learning_rate=0.01;, score=-1.047 total time=  24.2s
[CV 5/5; 10/10] START activation_function=linear, batch_size=10, dropout_rate=0.0, epochs=10, init=zero, learning_rate=0.01


C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


[CV 5/5; 10/10] END activation_function=linear, batch_size=10, dropout_rate=0.0, epochs=10, init=zero, learning_rate=0.01;, score=-0.987 total time=  24.0s


C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [41]:
print('Best : {}, using {}'.format(ran_result.best_score_,ran_result.best_params_))
means = ran_result.cv_results_['mean_test_score']
stds = ran_result.cv_results_['std_test_score']
params = ran_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : -0.1461825489997864, using {'learning_rate': 0.01, 'init': 'normal', 'epochs': 100, 'dropout_rate': 0.2, 'batch_size': 20, 'activation_function': 'tanh'}
-2.0077980279922487,0.047736334746462376 with: {'learning_rate': 0.01, 'init': 'normal', 'epochs': 50, 'dropout_rate': 0.2, 'batch_size': 10, 'activation_function': 'softmax'}
-0.9998110175132752,0.02510247507730764 with: {'learning_rate': 0.001, 'init': 'zero', 'epochs': 50, 'dropout_rate': 0.1, 'batch_size': 20, 'activation_function': 'tanh'}
-0.9996739387512207,0.025204382523166466 with: {'learning_rate': 0.1, 'init': 'zero', 'epochs': 100, 'dropout_rate': 0.1, 'batch_size': 20, 'activation_function': 'relu'}
-2.0077980279922487,0.047736334746462376 with: {'learning_rate': 0.1, 'init': 'uniform', 'epochs': 50, 'dropout_rate': 0.1, 'batch_size': 10, 'activation_function': 'softmax'}
-0.9996739387512207,0.025204382523166466 with: {'learning_rate': 0.001, 'init': 'zero', 'epochs': 10, 'dropout_rate': 0.2, 'batch_size': 20, 'act

In [57]:
def create_model():
    model = Sequential()
    model.add(Dense(11,input_dim = 11,kernel_initializer ='normal',input_shape=[10],activation ='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(6,kernel_initializer ='normal',input_shape=[10],activation ='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation ='tanh'))
    
    adam = Adam(lr =0.01)
    model.compile(loss = 'mse',optimizer = adam,metrics = ['accuracy'])
    return model

model=KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 20,epochs = 100)


C:\Users\HP\AppData\Local\Temp\ipykernel_8944\326674290.py:13: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 20,epochs = 100)


In [58]:
model.fit(x_train,y_train)

C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [66]:
y_pred=model.predict(x_train)
y_pred

array([-0.98442924, -0.9959265 ,  0.9709069 , ...,  0.98562986,
        0.9850236 ,  0.9967018 ], dtype=float32)